In [13]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
import glob as gl
import pickle as pic
import numpy as np
from music21 import converter, instrument, note, chord, pitch, stream
from keras.utils import np_utils

## Codificación de los datos de las canciones

In [14]:
notas = []

codificar = []

for archivo in gl.glob("canciones/*.mid"):
    cancion = converter.parse(archivo)
    
    aux = instrument.partitionByInstrument(cancion)
    
    codificar = aux.parts[0].recurse()
    
    for elemento in codificar:
        if isinstance(elemento, note.Note):
            n = str(elemento.pitch)
            notas.append(n)
        if isinstance(elemento, chord.Chord):
            notes = []
            for nota in elemento:
                notes.append(str(nota.pitch.midi))
            a = '.'.join(notes)
            notas.append(a)


## Generación de las secuencias para el entrenamiento

In [15]:
num_notas_distintas = len(set(notas))
tam_secuencia = 100

tonos = sorted(set(elemento for elemento in notas))
nota_a_int = dict((nota, numero) for numero, nota in enumerate(tonos))
int_a_nota = dict((numero, nota) for numero, nota in enumerate(tonos))

In [16]:
X = []
y = []

for i in range(0, len(notas) - tam_secuencia, 1):
    secuencia = notas[i: i+tam_secuencia]
    nota_sig = notas[i + tam_secuencia]
    X.append([nota_a_int[nota] for nota in secuencia])
    y.append(nota_a_int[nota_sig])

In [17]:
X_tr = np.reshape(X, (len(X), tam_secuencia, 1))


X_tr = X_tr / num_notas_distintas


y_tr = np_utils.to_categorical(y)

## Arquitectura de la red

In [18]:
model = Sequential()
model.add(LSTM(512,
               input_shape=(X_tr.shape[1], X_tr.shape[2]),
               recurrent_dropout=0.3,
               return_sequences=True))
model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
model.add(LSTM(512))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(num_notas_distintas))
model.add(Activation('softmax'))

In [19]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
lstm_5 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
lstm_6 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
batch_normalization_3 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
activation_3 (Activation)    (None, 256)              

## Compilación y entrenamiento de la red

In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [21]:
model.fit(X_tr, y_tr, epochs=50, batch_size=64)

Epoch 1/50
8035/8035 [==============================] - 446s 55ms/step - loss: 5.6642
Epoch 2/50
8035/8035 [==============================] - 428s 53ms/step - loss: 5.1317
Epoch 3/50
8035/8035 [==============================] - 429s 53ms/step - loss: 4.9870
Epoch 4/50
8035/8035 [==============================] - 429s 53ms/step - loss: 4.8107
Epoch 5/50
8035/8035 [==============================] - 429s 53ms/step - loss: 4.6460
Epoch 6/50
8035/8035 [==============================] - 429s 53ms/step - loss: 4.5166
Epoch 7/50
8035/8035 [==============================] - 429s 53ms/step - loss: 4.3816
Epoch 8/50
8035/8035 [==============================] - 428s 53ms/step - loss: 4.3000
Epoch 9/50
8035/8035 [==============================] - 429s 53ms/step - loss: 4.1715
Epoch 10/50
8035/8035 [==============================] - 431s 54ms/step - loss: 4.0265
Epoch 11/50
8035/8035 [==============================] - 430s 54ms/step - loss: 3.8749
Epoch 12/50
8035/8035 [=============================

## Guardamos el modelo generado

In [22]:
model.save('musica.h5')

## Generación de las notas que comformarán la melodía final (inversa de la codificación) utilizando la red entrenada

In [33]:
inicio = np.random.randint(0, len(X_tr-1))
secuencia = X[inicio]
composicion = []

# generamos notas
for indice in range(500):
    prediccion_in = np.reshape(secuencia, (1, len(secuencia), 1))
    prediccion_in = prediccion_in / float(num_notas_distintas)
    
    prediccion = model.predict(prediccion_in, verbose=0)

    index = np.argmax(prediccion)
    result = int_a_nota[index]
    composicion.append(result)

    secuencia.append(index)
    secuencia = secuencia[1:len(secuencia)]

## Generación del archivo MIDI final

In [34]:
offset = 0
cancion = []

for elemento in composicion:
    
    if ('.' in elemento) or elemento.isdigit():
        notas_acorde = elemento.split('.')
        notes = []
        for nota_actual in notas_acorde:
            new_nota = note.Note(int(nota_actual))
            new_nota.storedInstrument = instrument.Piano()
            notes.append(new_nota)
        new_acorde = chord.Chord(notes)
        new_acorde.offset = offset
        cancion.append(new_acorde)
    else:
        new_nota = note.Note(elemento)
        new_nota.offset = offset
        new_note.storedInstrument = instrument.Piano()
        cancion.append(new_note)

    offset += 0.5

midi_stream = stream.Stream(cancion)

midi_stream.write('midi', fp='cancion.mid')

'cancion.mid'